In [34]:
import random
from math import sin, cos

x_limit = [-1, 2]
y_limit = [-1, 1]

In [35]:
def generate_population():
    population = []

    for _ in range(6):
      kromosom = generate_kromosom()
      population.append(kromosom)

    return population

In [36]:
def generate_kromosom():
    kromosom = []

    for _ in range(6) :
      kromosom.append(random.randint(0,9))
    
    return kromosom

In [37]:
def decode(gen, limit) :
    kali, pembagi = 0, 0
    for i in range(len(gen)) :
        num = gen[i]
        kali += num * (10**-(i+1))
        pembagi += 9 * (10**-(i+1))

    return gen[0] + (((limit[1] - limit[0]) / pembagi) * kali) 

In [38]:
def split_kromosom(kromosom):
    split = len(kromosom) // 2
    return kromosom[:split], kromosom[split:] 

In [39]:
def function(x,y):
    return (cos(x**2) * sin(y**2)) + (x + y)

In [40]:
def best_population(population):
    for kromosom in population:
        maxfit = -999
        kromosom_a, kromosom_b = split_kromosom(kromosom)
        x1 = decode(kromosom_a, x_limit)
        x2 = decode(kromosom_b, y_limit)
        value = function(x1, x2)

        if  value > maxfit:
            maxfit = value
            max_kromosom = kromosom

    return max_kromosom

In [41]:
def parent_roulette_selection(population, fitness, fitness_total):
    r = random.random()
    i = 0
    while r > 0:
      r -= fitness[i]/fitness_total
      i += 1
      if (i == (len(population) - 1)) :
          break
          
    return population[i]

In [42]:
def crossover(parent_1, parent_2) :         
    child_1, child_2, childs = [], [], []
    pb = random.random()

    if pb < 0.9:
      child_1[:1], child_1[1:] = parent_1[:1], parent_2[1:]   
      child_2[:1], child_2[1:] = parent_2[:1], parent_1[1:]
      childs.append(child_1)
      childs.append(child_2)
    else:   
      childs.append(parent_1)
      childs.append(parent_2)

    return childs

In [43]:
def mutation(child_1, child_2):
    for i in range(len(child_1)):
        p = random.random()
        if p < 0.1:
           child_1[i] = random.randint(0,9)

    for i in range(len(child_2)):
        p = random.random()
        if p < 0.1:
           child_2[i] = random.randint(0,9)

    mutasi = []
    mutasi.append(child_1)
    mutasi.append(child_2)
    return mutasi

In [44]:
population_total = 6
generation = 3
population = generate_population()
print("Populasi Awal:", population, "\n")

for gen in range(generation):
      kromosom, fitness_data, new_population, best, child = [], [], [], [], []
      total_fitness = 0
      best = best_population(population)
      
      print('-----------------------------------------------------------------------------------------')
      print('Populasi Generasi', gen+1)
      print('-----------------------------------------------------------------------------------------')

      for i, kromosom in enumerate(population):                         
          kromosom_a, kromosom_b = split_kromosom(kromosom)
          x1 = decode(kromosom_a, x_limit)
          x2 = decode(kromosom_b, y_limit)

          fitness_value = function(x1, x2)
          fitness_data.append(fitness_value)
          total_fitness += fitness_value
          
          print(f'Kromosom {i+1}              = {kromosom}')
          print('Nilai Phenotype X1      =', x1)
          print('Nilai Phenotype X2      =', x2)
          print('Nilai Fungsi / Fitness  =', fitness_value, '\n')

      print('Total Nilai Fitness =', total_fitness)
      print('-----------------------------------------------------------------------------------------')
      print('\nSeleksi Orang Tua Dengan Roulette Wheel\n')

      for i in range(population_total // 2):
          parent_1 = parent_roulette_selection(population, fitness_data, total_fitness)
          parent_2 = parent_roulette_selection(population, fitness_data, total_fitness)
          print(f'Iterasi ke-{i+1}:')
          print('Hasil Seleksi Parent 1 = ', parent_1)
          print('Hasil Seleksi Parent 2 = ', parent_2)

          child = crossover(parent_1, parent_2)
          print('Hasil Crossover Child  = ', child, '\n')

          child = mutation(child[0], child[1])
          new_population.append(child[0])
          new_population.append(child[1])

      print('Populasi Baru Generasi', gen+2, ' = ', new_population)
      print('------------------------------------------------------------------------------------\n')

      best = best_population(new_population)
      population = new_population
      generation += 1

print('--------------------------------------------')
print('Hasil Akhir Kromosom Terbaik')
print('--------------------------------------------')
print('Kromosom                    = ', best)

a, b = split_kromosom(best)
x1 = decode(a, x_limit)
x2 = decode(b, y_limit)

print('Phenotype x1                = ', x1)
print('Phenotype x2                = ', x2)
print('Nilai Fungsi / Fitness      = ', function(x1,x2))
print('--------------------------------------------')

Populasi Awal: [[5, 0, 7, 7, 7, 0], [5, 8, 0, 7, 3, 7], [2, 9, 8, 8, 5, 9], [5, 7, 8, 9, 6, 4], [7, 4, 2, 0, 9, 9], [4, 5, 6, 8, 1, 2]] 

-----------------------------------------------------------------------------------------
Populasi Generasi 1
-----------------------------------------------------------------------------------------
Kromosom 1              = [5, 0, 7, 7, 7, 0]
Nilai Phenotype X1      = 6.5225225225225225
Nilai Phenotype X2      = 8.541541541541541
Nilai Fungsi / Fitness  = 14.979271590626078 

Kromosom 2              = [5, 8, 0, 7, 3, 7]
Nilai Phenotype X1      = 6.741741741741741
Nilai Phenotype X2      = 8.475475475475475
Nilai Fungsi / Fitness  = 15.25901527087129 

Kromosom 3              = [2, 9, 8, 8, 5, 9]
Nilai Phenotype X1      = 2.894894894894895
Nilai Phenotype X2      = 9.71971971971972
Nilai Fungsi / Fitness  = 12.502429759643865 

Kromosom 4              = [5, 7, 8, 9, 6, 4]
Nilai Phenotype X1      = 6.7357357357357355
Nilai Phenotype X2      = 10.9299